### Shapash Model Overview
##### With this tutorial you:
Understand how to create a Shapash SmartPredictor to make prediction and have local explanation in production with a simple use case.

This tutorial describes the different steps from training the model to Shapash SmartPredictor deployment. A more detailed tutorial allows you to know more about the SmartPredictor Object.

Contents:

- Build a Regressor
- Compile Shapash SmartExplainer
- From Shapash SmartExplainer to SmartPredictor
- Save Shapash Smartpredictor Object in pickle file
- Make a prediction

In [3]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
### Divide the dataset into independent and dependent dataset
y=df['tip']
X=df[df.columns.difference(['tip'])]
X.head()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.3 KB


### Encode the categorical features:

In [7]:
X['day']=X['day'].cat.codes
X['sex']=X['sex'].cat.codes
X['smoker']=X['smoker'].cat.codes
X['time']=X['time'].cat.codes

<ipython-input-7-94ad230a0023>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['day']=X['day'].cat.codes
<ipython-input-7-94ad230a0023>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['sex']=X['sex'].cat.codes
<ipython-input-7-94ad230a0023>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [8]:
X

,day,sex,size,smoker,time,total_bill
0,3,1,2,1,1,16.99
1,3,0,3,1,1,10.34
2,3,0,3,1,1,21.01
3,3,0,2,1,1,23.68
4,3,1,4,1,1,24.59
...,...,...,...,...,...,...
239,2,0,3,1,1,29.03
240,2,1,2,0,1,27.18
241,2,0,2,0,1,22.67
242,2,0,2,1,1,17.82


In [9]:
#Train, test split and model fitting:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=1)

In [10]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=200).fit(X_train,y_train)

#### Lets Understand Our Model With Shapash 
In this section, we use the SmartExplainer Object from shapash.

- It allows users to understand how the model works with the specified data.
- This object must be used only for data mining step. Shapash provides another object for deployment.


In [11]:
!pip install shapash

In [12]:
from shapash.explainer.smart_explainer import SmartExplainer

In [13]:
xpl = SmartExplainer()

In [14]:
xpl.compile(
    x=X_test,
    model=regressor,
   
)

Backend: Shap TreeExplainer


In [15]:
xpl

#### Lets Understand the results of your trained model
Then, we can easily get a first summary of the explanation of the model results.

- Here, we chose to get the 3 most contributive features for each prediction.
- We used a wording to get features names more understandable in operationnal case.

In [16]:
app = xpl.run_app(title_story='Tips Dataset')

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
/home/aajay/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:root:Your Shapash application run on http://aajay:8050/
INFO:root:Use the method .kill() to down your app.


Dash is running on http://0.0.0.0:8050/



INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)
 * Environment: production


In [17]:
predictor = xpl.to_smartpredictor()

   Use a production WSGI server instead.


In [26]:
#Save Using PKL
predictor.save('./predictor.pkl')

INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:00:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:00:28] "GET /assets/material-icons.css?m=1619271350.424513 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:00:28] "GET /assets/style.css?m=1619271350.4285116 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:00:28] "GET /_dash-component-suites/dash_renderer/react@16.v1_8_3m1619271347.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:00:28] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_8_3m1619271347.8.7.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:00:28] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_8_3m1619271347.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:00:28] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_8_3m1619271347.7.2.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:00:28] "GET /_dash-component-suites/dash_core_c

INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:01:32] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:01:34] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:01:34] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:01:34] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:01:34] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:04:56] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:04:56] "POST /_dash-update-component HTTP/1.1" 204 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:04:56] "POST /_dash-update-component HTTP/1.1" 204 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:04:56] "POST /_dash-update-component HTTP/1.1" 204 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2021 22:04:56] "POST /_dash-update-component HTTP/1.1" 204 -
INFO:werkzeug:127.0.0.1 - - [2

In [19]:
from shapash.utils.load_smartpredictor import load_smartpredictor
predictor_load = load_smartpredictor('./predictor.pkl')

#### Make a prediction with your SmartPredictor
In order to make new predictions and summarize local explainability of your model on new datasets, you can use the method add_input of the SmartPredictor.

- The add_input method is the first step to add a dataset for prediction and explainability.
- It checks the structure of the dataset, the prediction and the contribution if specified.
- It applies the preprocessing specified in the initialisation and reorder the features with the order used by the model. (see the documentation of this method)
- In API mode, this method can handle dictionnaries data which can be received from a GET or a POST request.
- Add data
- The x input in add_input method doesn't have to be encoded, add_input applies preprocessing.

In [27]:
predictor_load.add_input(x=X, ypred=y)
detailed_contributions = predictor_load.detail_contributions()
detailed_contributions.head()

,tip,day,sex,size,smoker,time,total_bill
0,1.01,0.065553,0.103861,-0.016624,-0.030139,-0.006143,0.124617
1,1.66,0.102582,-0.046297,-0.006904,-0.091398,-0.018797,-1.255661
2,3.50,0.074176,-0.024642,-0.010637,-0.008496,0.004780,0.394944
3,3.31,0.134599,-0.023065,-0.036672,0.007530,0.004714,0.031819
4,3.61,0.032535,0.060528,-0.035247,-0.005128,-0.006684,-0.248980


#### Summarize explanability of the predictions
- You can use the summarize method to summarize your local explainability
- This summary can be configured with modify_mask method so that you have explainability that meets your operational needs.

In [28]:
predictor_load.modify_mask(max_contrib=3)
explanation = predictor_load.summarize()
explanation.head()

,tip,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
0,1.01,total_bill,16.99,0.124617,sex,1,0.103861,day,3,0.0655533
1,1.66,total_bill,10.34,-1.25566,day,3,0.102582,smoker,1,-0.0913977
2,3.50,total_bill,21.01,0.394944,day,3,0.0741759,sex,0,-0.0246418
3,3.31,day,3,0.134599,size,2,-0.036672,total_bill,23.68,0.0318187
4,3.61,total_bill,24.59,-0.24898,sex,1,0.0605282,size,4,-0.0352466
